In [1]:
import keras.layers as layers
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from keras.models import Model
import keras.optimizers as optim

from game.game import Game
from utils.dataprep import load_emb_gz, make_categories

Using TensorFlow backend.


In [2]:
DATASET = "data/imagenet-200x65-vgg19.train.emb.gz"

N_IMAGES = 2
EMBEDDING_SIZE = 50
VOCABULARY_SIZE = 10

TEMPERATURE = 10.0
N_EPISODES = 2000
ANALYSIS_WINDOW = 20

OPTIMIZER = optim.Adam(0.05)
# OPTIMIZER = optim.SGD(2)

In [3]:
path2ind, path_list, embeddings = load_emb_gz(DATASET)
categories = make_categories(path_list)
IMAGE_SHAPE = [len(embeddings[0])]

DONE. 13190 items loaded from data/imagenet-200x65-vgg19.train.emb.gz.
200 categories found.


In [4]:
game = Game(images=embeddings, images_filenames=path_list, categories=categories)

In [5]:
def getshape(array):
    if isinstance(array, list):
        return [getshape(x) for x in array]
    else:
        try:
            return array.shape
        except:
            return None

In [6]:
class Sender:
    def __init__(self, n_images, input_image_shape, embedding_size, vocabulary_size, temperature):
        self.build_model(n_images, input_image_shape, embedding_size, vocabulary_size, temperature)
        
    def build_model(self, n_images, input_image_shape, embedding_size, vocabulary_size, temperature):
        image_inputs = [layers.Input(shape=input_image_shape, name=f"S_image_in_{i}") for i in range(n_images)]
        image_embedding_layer = layers.Dense(embedding_size, name="S_image_embedding")
        sigmoid = layers.Activation("sigmoid", name="S_sigmoid")
        concatenate = layers.Concatenate(axis=-1)
        output_layer = layers.Dense(vocabulary_size, name="S_output")
        temperature = layers.Lambda(lambda x: x / temperature, name="S_temperature")
        softmax = layers.Softmax()
        
        y = [image_embedding_layer(x) for x in image_inputs]
        y = [sigmoid(x) for x in y]
        y = concatenate(y)
        y = output_layer(y)
        y = temperature(y)
        y = softmax(y)
        
        self.model = Model(image_inputs, y, name="S_predict")
        
        index = layers.Input(shape=[1], dtype="int32", name="S_index_in")
        y_selected = layers.Lambda(
#             lambda probs_index: K.gather(probs_index[0][0], probs_index[1]),
            lambda probs_index: tf.gather(*probs_index, axis=-1),
            name="S_gather")([y, index])
        
        
        def loss(target, prediction):
            return - K.log(prediction) * target
        
        self.model_train = Model([*image_inputs, index], y_selected, name="S_train")
        self.model_train.compile(loss=loss, optimizer=OPTIMIZER)

        self.model.summary()
        self.model_train.summary()
        
    def predict(self, state):
#         print(getshape(state))
        return self.model.predict_on_batch(x=state)

    def update(self, state, action, target):
        x = [*state, action]
        print(getshape(x), getshape(target))
        return self.model_train.train_on_batch(x=x, y=target)

In [7]:
class Receiver:
    def __init__(self, n_images, input_image_shape, embedding_size, vocabulary_size, temperature):
        self.build_model(n_images, input_image_shape, embedding_size, vocabulary_size, temperature)
        
    def build_model(self, n_images, input_image_shape, embedding_size, vocabulary_size, temperature):
        image_inputs = [layers.Input(shape=input_image_shape, name=f"R_image_in_{i}") for i in range(n_images)]
        image_embedding_layer = layers.Dense(embedding_size, name="R_image_embedding")
        temperature = layers.Lambda(lambda x: x / temperature, name="R_temperature")
        softmax = layers.Softmax()
        
        symbol_input = layers.Input(shape=[1], dtype="int32", name=f"R_symbol_in")
        symbol_embedding = layers.Embedding(input_dim=vocabulary_size, 
                                            output_dim=embedding_size, 
                                            name="R_symbol_embedding")
        dot_product = layers.Dot(axes=-1, name="R_dot_product")
        concatenate = layers.Concatenate(axis=-1)
        
        y_images = [image_embedding_layer(x) for x in image_inputs]
        y_symbol = symbol_embedding(symbol_input)
        y = [dot_product([img, y_symbol]) for img in y_images]
        y = concatenate(y)
        y = temperature(y)
        y = softmax(y)
        
        self.model = Model([*image_inputs, symbol_input], y, name="R_predict")
        
        index = layers.Input(shape=[1], dtype="int32", name="R_index_in")
        y_selected = layers.Lambda(
#             lambda probs_index: K.gather(probs_index[0][0], probs_index[1]),
            lambda probs_index: tf.gather(*probs_index, axis=-1),
            name="R_gather")([y, index])
        
        
        def loss(target, prediction):
            return - K.log(prediction) * target
        
        self.model_train = Model([*image_inputs, symbol_input, index], y_selected, name="R_train")
        self.model_train.compile(loss=loss, optimizer=OPTIMIZER)

        self.model.summary()
        self.model_train.summary()
        
    def predict(self, state):
#         print(getshape(state))
        return self.model.predict_on_batch(x=state)

    def update(self, state, action, target):
        x = [*state, action]
        print(getshape(x), getshape(target))
        return self.model_train.train_on_batch(x=x, y=target)

In [8]:
K.clear_session()

sender = Sender(n_images=N_IMAGES, 
                input_image_shape=IMAGE_SHAPE, 
                embedding_size=EMBEDDING_SIZE, 
                vocabulary_size=VOCABULARY_SIZE, 
                temperature=TEMPERATURE)
receiver = Receiver(n_images=N_IMAGES, 
                    input_image_shape=IMAGE_SHAPE, 
                    embedding_size=EMBEDDING_SIZE, 
                    vocabulary_size=VOCABULARY_SIZE, 
                    temperature=TEMPERATURE)

TypeError: Failed to convert object of type <class 'keras.layers.core.Lambda'> to Tensor. Contents: <keras.layers.core.Lambda object at 0x00000166E7C7B988>. Consider casting elements to a supported type.

In [ ]:
success_log = []
sender_log = []
receiver_log = []

for episode in range(1, N_EPISODES + 1):
    game.reset()
    sender_state = game.get_sender_state(n_images=N_IMAGES, unique_categories=True, expand=True)
    sender_probs = sender.predict(state=sender_state)
    sender_probs = np.squeeze(sender_probs)
    sender_action = np.random.choice(np.arange(len(sender_probs)), p=sender_probs)
#     sender_action = 0
    
    receiver_state = game.get_receiver_state(sender_action, expand=True)
    receiver_probs = receiver.predict(receiver_state)
    receiver_probs = np.squeeze(receiver_probs)
    receiver_action = np.random.choice(np.arange(len(receiver_probs)), p=receiver_probs)
    
#     receiver_action = 0
    sender_reward, receiver_reward, success = game.evaluate_guess(receiver_action)

    sender_loss = sender.update(sender_state, np.asarray([sender_action]), np.asarray([sender_reward]))
    receiver_loss = receiver.update(receiver_state, np.asarray([receiver_action]), np.asarray([receiver_reward]))
    
    success_log.append(success)
#     sender_log.append(sender_loss)
#     receiver_log.append(receiver_loss)
    
    if not episode % 10:
        print(f"ep {episode}, \
        success_rate: {sum(success_log[-ANALYSIS_WINDOW:])/ANALYSIS_WINDOW}, \
        sender_loss: {sum(sender_log[-ANALYSIS_WINDOW:])/ANALYSIS_WINDOW}, \
        receiver_loss: {sum(receiver_log[-ANALYSIS_WINDOW:])/ANALYSIS_WINDOW}".
        replace("    ", ""))


In [ ]:
sender_state = [np.random.random([1, 1000]) for _ in ".."]
sender.update(sender_state, np.asarray([0]), np.asarray([0]))

In [ ]:
plt.plot(np.convolve(success_log, np.ones((40,))/40, mode='valid'), label="Average success rate");
plt.legend()
plt.title("Success rate");

In [ ]:
plt.plot(sender_log, linestyle="None", marker=".", label="Loss")
plt.plot(np.convolve(sender_log, np.ones((40,))/40, mode='valid'), label="Average loss")
plt.legend()
plt.title("Sender loss");

In [ ]:
plt.plot(receiver_log, linestyle="None", marker=".", label="Loss")
plt.plot(np.convolve(receiver_log, np.ones((40,))/40, mode='valid'), label="Average loss")
plt.legend()
plt.title("Receiver loss");